# Partitioning

In [113]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [114]:
clean_9000 = 'result_clean_pdf_9000_files.csv'
clean_109 = 'result_clean_pdf_109_embed_video.csv'
malw_173 = 'result_malware_pdf_cve_sorted_173_files.csv'
malw_10982 = 'result_malware_pdf_pre_04-2011_10982_files.csv'
df_clean_9000 = pd.read_csv(clean_9000, sep=',')
df_clean_109 = pd.read_csv(clean_109, sep=',')
df_malw_173 = pd.read_csv(malw_173, sep=',')
df_malw_10982 = pd.read_csv(malw_10982, sep=',')

In [147]:
df_clean = pd.concat([df_clean_9000, df_clean_109], axis=0, ignore_index=True)
df_malw = pd.concat([df_malw_10982, df_malw_173], axis=0, ignore_index=True)

df_clean["malware"] = np.zeros(df_clean.shape[0], dtype='bool')
df_malw["malware"] = np.ones(df_malw.shape[0], dtype='bool')

In [148]:
df_tot = pd.concat([df_clean, df_malw], axis=0, ignore_index=True)

df_train, df_test = train_test_split(df_tot, test_size=0.1)
df_train, df_val = train_test_split(df_train, test_size=0.1)

df_train.to_csv("df_train.csv", index = False)
df_val.to_csv("df_val.csv", index = False)
df_test.to_csv("df_test.csv", index = False)

In [149]:
label = 'malware'
#We copy the output label
y_train=df_train[label]
#We remove the output label from X
X_train=df_train.drop([label],axis=1)

#process is repeated for test and validation
y_test=df_test[label]
X_test=df_test.drop([label],axis=1)

y_val=df_val[label]
X_val=df_val.drop([label],axis=1)

#We call prepare_data
#X_train = np.array(X_train)
#X_val = np.array(X_val)
#X_test = np.array(X_test)
#y_train = np.array(y_train)
#y_val = np.array(y_val)
#y_test = np.array(y_test)

In [150]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
y_hat = knn.predict(X_test)
#print(y_hat.sum() / df_test.shape[0]==True)
#for i in range(y_ha)
pred = y_test == y_hat
pred.sum() / len(y_test)

0.9965363681345868